In [49]:
import pandas as pd
import numpy as np
from IPython.core import display as ICD

In [4]:
group = pd.read_csv('group12_annotation.csv', encoding = 'utf8', index_col=0)
group.head()

,id,annotator,rating
0,t3_n28jtz,user1,5
1,t3_n28jtz,user2,2
2,t3_n28jtz,user3,5
3,t3_n2bkyt,user1,5
4,t3_n2bkyt,user2,4


In [6]:
group_pivot = group.pivot(index='id', columns='annotator', values='rating')
group_pivot.head()

annotator,user1,user2,user3
id,,,
t3_n28jtz,5,2,5
t3_n2bkyt,5,4,3
t3_n2eqpj,5,5,5
t3_n2ixa0,4,3,3
t3_n2m2ws,2,2,2


In [7]:
group_pivot.corr()

annotator,user1,user2,user3
annotator,,,
user1,1.000000,0.767748,0.797113
user2,0.767748,1.000000,0.780097
user3,0.797113,0.780097,1.000000


In [9]:
def nominal_metric(a, b):
    return a != b


def interval_metric(a, b):
    return (a-b)**2


def ratio_metric(a, b):
    return ((a-b)/(a+b))**2

In [10]:
def krippendorff_alpha(data, metric=interval_metric, force_vecmath=False, convert_items=float, missing_items=None):
    '''
    Calculate Krippendorff's alpha (inter-rater reliability):
    
    data is in the format
    [
        {unit1:value, unit2:value, ...},  # coder 1
        {unit1:value, unit3:value, ...},   # coder 2
        ...                            # more coders
    ]
    or 
    it is a sequence of (masked) sequences (list, numpy.array, numpy.ma.array, e.g.) with rows corresponding to coders and columns to items
    
    metric: function calculating the pairwise distance
    force_vecmath: force vector math for custom metrics (numpy required)
    convert_items: function for the type conversion of items (default: float)
    missing_items: indicator for missing items (default: None)
    '''
    
    # number of coders
    m = len(data)
    
    # set of constants identifying missing values
    if missing_items is None:
        maskitems = []
    else:
        maskitems = list(missing_items)
    if np is not None:
        maskitems.append(np.ma.masked_singleton)
    
    # convert input data to a dict of items
    units = {}
    for d in data:
        try:
            # try if d behaves as a dict
            diter = d.items()
        except AttributeError:
            # sequence assumed for d
            diter = enumerate(d)
            
        for it, g in diter:
            if g not in maskitems:
                try:
                    its = units[it]
                except KeyError:
                    its = []
                    units[it] = its
                its.append(convert_items(g))


    units = dict((it, d) for it, d in units.items() if len(d) > 1)  # units with pairable values
    n = sum(len(pv) for pv in units.values())  # number of pairable values
    
    if n == 0:
        raise ValueError("No items to compare.")
    
    np_metric = (np is not None) and ((metric in (interval_metric, nominal_metric, ratio_metric)) or force_vecmath)
    
    Do = 0.
    for grades in units.values():
        if np_metric:
            gr = np.asarray(grades)
            Du = sum(np.sum(metric(gr, gri)) for gri in gr)
        else:
            Du = sum(metric(gi, gj) for gi in grades for gj in grades)
        Do += Du/float(len(grades)-1)
    Do /= float(n)

    if Do == 0:
        return 1.

    De = 0.
    for g1 in units.values():
        if np_metric:
            d1 = np.asarray(g1)
            for g2 in units.values():
                De += sum(np.sum(metric(d1, gj)) for gj in g2)
        else:
            for g2 in units.values():
                De += sum(metric(gi, gj) for gi in g1 for gj in g2)
    De /= float(n*(n-1))

    return 1.-Do/De if (Do and De) else 1.

In [16]:
data = [list(group_pivot['user1'].values), list(group_pivot['user2'].values), list(group_pivot['user3'].values)]

print("nominal metric: %.3f" % krippendorff_alpha(data, nominal_metric))
print("interval metric: %.3f" % krippendorff_alpha(data, interval_metric))

nominal metric: 0.524
interval metric: 0.779


In [20]:
ids = list(set(group['id'].values))

In [22]:
total = pd.read_csv('si630w22-hw3-train.csv', encoding = 'utf8')
total.head()

,id,annotator_id,rating,group
0,t3_n27vu3,user_00,5.0,group_09
1,t3_n27vu3,user_01,5.0,group_09
2,t3_n27vu3,user_02,5.0,group_09
3,t3_n2az7m,user_00,5.0,group_09
4,t3_n2az7m,user_01,5.0,group_09


In [30]:
other = total[(total['id'].isin(ids)) & (total['group'] != 'group_12')]
other.head()

,id,annotator_id,rating,group
24,t3_n2xr6w,user_00,5.0,group_09
25,t3_n2xr6w,user_01,5.0,group_09
26,t3_n2xr6w,user_02,5.0,group_09
72,t3_n49fjq,user_00,1.0,group_09
73,t3_n49fjq,user_01,2.0,group_09


In [41]:
groups = list(set(other['group'].values))
user_dict = {}
for group in groups:
    user_dict[group] = list(set(other[other['group'] == group]['annotator_id']))
user_dict

{'group_03': ['user_03', 'user_04'],
 'group_10': ['user_05', 'user_07', 'user_06'],
 'group_11': ['user_56', 'user_58', 'user_57'],
 'group_17': ['user_32', 'user_33', 'user_31'],
 'group_14': ['user_54', 'user_55'],
 'group_09': ['user_00', 'user_01', 'user_02'],
 'group_08': ['user_39', 'user_38'],
 'group_22': ['user_12', 'user_11', 'user_10'],
 'group_02': ['user_16', 'user_17', 'user_15'],
 'group_23': ['user_51', 'user_50'],
 'group_19': ['user_37', 'user_36'],
 'group_16': ['user_48', 'user_49'],
 'group_05': ['user_52', 'user_53'],
 'group_07': ['user_40', 'user_42', 'user_41'],
 'group_01': ['user_13', 'user_14'],
 'group_24': ['user_29', 'user_30', 'user_28'],
 'group_15': ['user_27', 'user_26'],
 'group_13': ['user_09', 'user_08'],
 'group_21': ['user_47', 'user_46'],
 'group_25': ['user_35', 'user_34'],
 'group_04': ['user_21', 'user_22'],
 'group_18': ['user_43', 'user_44', 'user_45'],
 'group_20': ['user_19', 'user_18', 'user_20']}

In [45]:
other_pivot = other.pivot(index='id', columns='annotator_id', values='rating')
other_pivot.head()

annotator_id,user_00,user_01,user_02,user_03,user_04,user_05,user_06,user_07,user_08,user_09,...,user_49,user_50,user_51,user_52,user_53,user_54,user_55,user_56,user_57,user_58
id,,,,,,,,,,,,,,,,,,,,,
t3_n28jtz,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t3_n2bkyt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t3_n2m2ws,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,2.0,NaN,NaN,NaN,NaN,NaN
t3_n2rc8x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t3_n2ufls,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
for group in user_dict:
    ICD.display(group)
    print(other_pivot[user_dict[group]].corr())

'group_03'

annotator_id   user_03   user_04
annotator_id                    
user_03       1.000000  0.721688
user_04       0.721688  1.000000


'group_10'

annotator_id   user_05   user_07   user_06
annotator_id                              
user_05       1.000000  0.667246  0.498964
user_07       0.667246  1.000000  0.164276
user_06       0.498964  0.164276  1.000000


'group_11'

...

annotator_id  user_21  user_22
annotator_id                  
user_21       1.00000  0.57783
user_22       0.57783  1.00000


'group_18'

annotator_id  user_43   user_44   user_45
annotator_id                             
user_43           NaN       NaN       NaN
user_44           NaN  1.000000  0.431331
user_45           NaN  0.431331  1.000000


'group_20'

annotator_id   user_19   user_18   user_20
annotator_id                              
user_19       1.000000  0.711623  0.844742
user_18       0.711623  1.000000  0.638172
user_20       0.844742  0.638172  1.000000


In [51]:
other_pivot_fillna = other_pivot.fillna('*')
other_pivot_fillna.head()

annotator_id,user_00,user_01,user_02,user_03,user_04,user_05,user_06,user_07,user_08,user_09,...,user_49,user_50,user_51,user_52,user_53,user_54,user_55,user_56,user_57,user_58
id,,,,,,,,,,,,,,,,,,,,,
t3_n28jtz,*,*,*,*,*,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*
t3_n2bkyt,*,*,*,*,*,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*
t3_n2m2ws,*,*,*,*,*,*,*,*,*,*,...,*,*,*,3.0,2.0,*,*,*,*,*
t3_n2rc8x,*,*,*,*,*,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*
t3_n2ufls,*,*,*,*,*,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*


In [52]:
data2 = [list(other_pivot_fillna[user].values) for user in other_pivot_fillna.columns]
data2

[['*',
  '*',
  '*',
  '*',
  '*',
  5.0,
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  1.0,
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  1.0,
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  1.0,
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  3.0,
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  3.0,
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  5.0,
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  5.0,
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',
  '*',

In [53]:
missing = '*'
print("nominal metric: %.3f" % krippendorff_alpha(data2, nominal_metric, missing_items=missing))
print("interval metric: %.3f" % krippendorff_alpha(data2, interval_metric, missing_items=missing))

nominal metric: 0.426
interval metric: 0.689


In [55]:
score_dict = {'group':[], 'nominal': [], 'ordinal': []}
for group in user_dict:
    group_data = [list(other_pivot_fillna[user].values) for user in user_dict[group]]
    print(group)
    score_dict['group'].append(group)
    score_dict['nominal'].append(krippendorff_alpha(group_data, nominal_metric, missing_items=missing))
    score_dict['ordinal'].append(krippendorff_alpha(group_data, interval_metric, missing_items=missing))
    print("nominal metric: %.3f" % krippendorff_alpha(group_data, nominal_metric, missing_items=missing))
    print("interval metric: %.3f" % krippendorff_alpha(group_data, interval_metric, missing_items=missing))

group_03
nominal metric: 0.299
interval metric: 0.730
group_10
nominal metric: 0.065
interval metric: 0.062
group_11
nominal metric: 0.129
interval metric: 0.038
group_17
nominal metric: 0.073
interval metric: 0.264
group_14
nominal metric: 0.442
interval metric: 0.561
group_09
nominal metric: 0.827
interval metric: 0.915
group_08
nominal metric: -0.305
interval metric: -0.407
group_22
nominal metric: 0.039
interval metric: 0.458
group_02
nominal metric: 0.274
interval metric: 0.296
group_23
nominal metric: 1.000
interval metric: 1.000
group_19
nominal metric: 0.893
interval metric: 0.887
group_16
nominal metric: 0.194
interval metric: 0.550
group_05
nominal metric: 0.133
interval metric: 0.489
group_07
nominal metric: 0.312
interval metric: 0.331
group_01
nominal metric: 0.254
interval metric: 0.675
group_24
nominal metric: 0.437
interval metric: 0.710
group_15
nominal metric: 0.863
interval metric: 0.944
group_13
nominal metric: 0.425
interval metric: 0.819
group_21
nominal metric: 0

In [57]:
pd.DataFrame(data=score_dict).sort_values(by=['group'])

,group,nominal,ordinal
14,group_01,0.254144,0.675410
8,group_02,0.274390,0.296482
0,group_03,0.298969,0.730159
20,group_04,0.210784,0.593197
12,group_05,0.133028,0.488636
13,group_07,0.311978,0.331481
6,group_08,-0.304833,-0.406607
5,group_09,0.826858,0.914960
1,group_10,0.064615,0.062382
2,group_11,0.129129,0.037555


In [58]:
total2 = total[total['id'].isin(ids)]
total2.head()

,id,annotator_id,rating,group
24,t3_n2xr6w,user_00,5.0,group_09
25,t3_n2xr6w,user_01,5.0,group_09
26,t3_n2xr6w,user_02,5.0,group_09
72,t3_n49fjq,user_00,1.0,group_09
73,t3_n49fjq,user_01,2.0,group_09


In [60]:
mean_rating = total2.groupby(['id', 'group']).mean()
mean_rating.head()

rating
id        group             
t3_n28jtz group_01  5.000000
          group_12  4.000000
t3_n2bkyt group_02  4.666667
          group_12  4.000000
t3_n2m2ws group_05  2.500000

In [76]:
diff_rating = mean_rating.groupby(['id']).diff().dropna()
diff_rating['rating'] = abs(diff_rating['rating'])
top10_diff = diff_rating.sort_values(by=['rating'], ascending=False).head(10).reset_index(level=[0,1])[['id', 'rating']]
top10_diff.columns = ['id', 'diff_rating']
top10_diff

,id,diff_rating
0,t3_nir04f,3.333333
1,t3_nlsfqo,3.333333
2,t3_nl6dx3,2.833333
3,t3_nl4icj,2.666667
4,t3_ng8b9p,2.666667
5,t3_njrkw6,2.500000
6,t3_nai64u,2.500000
7,t3_n5lm2s,2.333333
8,t3_nd4zey,2.166667
9,t3_nmnfge,2.000000


In [78]:
top10_diff.merge(mean_rating.reset_index(level=[0,1]), how='left', on='id')

,id,diff_rating,group,rating
0,t3_nir04f,3.333333,group_10,4.666667
1,t3_nir04f,3.333333,group_12,1.333333
2,t3_nlsfqo,3.333333,group_01,5.000000
3,t3_nlsfqo,3.333333,group_12,1.666667
4,t3_nl6dx3,2.833333,group_08,1.500000
5,t3_nl6dx3,2.833333,group_12,4.333333
6,t3_nl4icj,2.666667,group_07,4.666667
7,t3_nl4icj,2.666667,group_12,2.000000
8,t3_ng8b9p,2.666667,group_12,2.333333
9,t3_ng8b9p,2.666667,group_21,5.000000
